<a href="https://colab.research.google.com/github/ammn23/car-image/blob/main/car_image_model_train_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uH2mHh3tXWxKwnKP6SlE")
project = rf.workspace("vipin-kumar-9fbyf").project("dent-d2fru")
version = project.version(1)
dataset = version.download("yolov8")

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uH2mHh3tXWxKwnKP6SlE")
project = rf.workspace("rahul-bv2nh").project("dent-eduux")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Dent--1 in yolov8:: 100%|██████████| 5208/5208 [00:01<00:00, 4454.94it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to DENT-1 in yolov8:: 100%|██████████| 1186/1186 [00:00<00:00, 10376.70it/s]


In [ ]:

import os
import shutil
import random
import yaml

# --- CONFIGURATION ---

# 1. Define the final class structure (to maintain consistency)
TARGET_CLASSES = {'dirt': 0, 'scratch': 1, 'dent': 2, 'rust': 3, 'clean': 4}

# 2. Define rules for processing your new datasets
PROCESSING_RULES = {
    'DENT-1':  {'source_class_id': 2, 'target_class_name': 'scratch'},
    'Dent--1': {'source_class_id': 1, 'target_class_name': 'dent'}
}

# 3. Define paths
BASE_PATH = '/content/'
DEST_PATH = os.path.join(BASE_PATH, 'FINAL_NEW_DATASET')
TEMP_MERGE_PATH = os.path.join(BASE_PATH, 'TEMP_MERGED_DATA')

# 4. Define the train, validation, and test split ratio
SPLIT_RATIO = {'train': 0.8, 'val': 0.2}

# --- SCRIPT LOGIC ---

def process_and_merge(source_folder, dest_folder, rule):
    source_labels_path = os.path.join(BASE_PATH, source_folder, 'train', 'labels')
    source_images_path = os.path.join(BASE_PATH, source_folder, 'train', 'images')

    dest_labels_path = os.path.join(dest_folder, 'labels')
    dest_images_path = os.path.join(dest_folder, 'images')

    source_class_id_str = str(rule['source_class_id'])
    target_class_id_str = str(TARGET_CLASSES[rule['target_class_name']])

    print(f"--- Merging '{source_folder}' ---")

    if not os.path.isdir(source_labels_path):
        print(f"Warning: Label directory not found at {source_labels_path}. Skipping.")
        return

    for label_filename in os.listdir(source_labels_path):
        if not label_filename.endswith('.txt'): continue

        original_label_path = os.path.join(source_labels_path, label_filename)
        new_label_lines = []

        with open(original_label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts and parts[0] == source_class_id_str:
                    parts[0] = target_class_id_str
                    new_label_lines.append(" ".join(parts))

        if new_label_lines:
            dest_label_path = os.path.join(dest_labels_path, label_filename)
            with open(dest_label_path, 'w') as f:
                f.write("\n".join(new_label_lines))

            image_filename = os.path.splitext(label_filename)[0] + '.jpg'
            source_image_path = os.path.join(source_images_path, image_filename)
            dest_image_path = os.path.join(dest_images_path, image_filename)
            if os.path.exists(source_image_path):
                shutil.copy(source_image_path, dest_image_path)

if __name__ == "__main__":
    # 1. Cleanup and setup directories
    if os.path.exists(DEST_PATH): shutil.rmtree(DEST_PATH)
    if os.path.exists(TEMP_MERGE_PATH): shutil.rmtree(TEMP_MERGE_PATH)
    os.makedirs(os.path.join(TEMP_MERGE_PATH, 'images'), exist_ok=True)
    os.makedirs(os.path.join(TEMP_MERGE_PATH, 'labels'), exist_ok=True)

    # 2. Process and merge new datasets into the temporary folder
    for folder_name, rule in PROCESSING_RULES.items():
        process_and_merge(folder_name, TEMP_MERGE_PATH, rule)

    # --- MODIFICATION: The step to copy the initial_dataset has been removed. ---

    # 3. Split the merged data into train, val, and test sets
    print("\n--- Splitting data into train, val, and test sets ---")
    all_images = sorted(os.listdir(os.path.join(TEMP_MERGE_PATH, 'images')))

    if not all_images:
        print("No images were found after processing. Exiting.")
    else:
        random.shuffle(all_images)

        total_files = len(all_images)
        train_end = int(total_files * SPLIT_RATIO['train'])
        val_end = train_end + int(total_files * SPLIT_RATIO['val'])

        splits = {
            'train': all_images[:train_end],
            'val': all_images[train_end:val_end]
        }

        for split_name, file_list in splits.items():
            print(f"Creating '{split_name}' set with {len(file_list)} images.")
            for image_name in file_list:
                label_name = os.path.splitext(image_name)[0] + '.txt'

                os.makedirs(os.path.join(DEST_PATH, split_name, 'images'), exist_ok=True)
                os.makedirs(os.path.join(DEST_PATH, split_name, 'labels'), exist_ok=True)

                shutil.move(os.path.join(TEMP_MERGE_PATH, 'images', image_name), os.path.join(DEST_PATH, split_name, 'images', image_name))
                shutil.move(os.path.join(TEMP_MERGE_PATH, 'labels', label_name), os.path.join(DEST_PATH, split_name, 'labels', label_name))

        # 4. Generate the final data.yaml file
        print("\n--- Generating final data.yaml ---")
        yaml_content = {
            'path': os.path.abspath(DEST_PATH),
            'train': 'train/images',
            'val': 'val/images',
            'nc': len(TARGET_CLASSES),
            'names': list(TARGET_CLASSES.keys())
        }

        with open(os.path.join(DEST_PATH, 'data.yaml'), 'w') as f:
            yaml.dump(yaml_content, f, sort_keys=False)

        print(f"\n✅ All done! Your new dataset is ready in '{DEST_PATH}'")

    # 5. Clean up temporary directory
    shutil.rmtree(TEMP_MERGE_PATH)

--- Merging 'DENT-1' ---
--- Merging 'Dent--1' ---

--- Splitting data into train, val, and test sets ---
Creating 'train' set with 1349 images.
Creating 'val' set with 337 images.

--- Generating final data.yaml ---

✅ All done! Your new dataset is ready in '/content/FINAL_NEW_DATASET'


In [ ]:
!rm -rf "/content/FINAL_NEW_DATASET"

In [ ]:
!pip install -q ultralytics

# Import the YOLO class
from ultralytics import YOLO
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO

# Load your custom-trained model
# This is the key step for fine-tuning. Instead of a generic model
# like 'yolov8n.pt', you load your own 'best.pt'.
model = YOLO('/content/best.pt')

# Fine-tune the model with the new dataset
# You might not need as many epochs as the initial training.
results = model.train(
    data='/content/FINAL_NEW_DATASET/data.yaml',          # Path to the .yaml file for your new data
    epochs=50,                     # Number of epochs to train for
    imgsz=640,                     # Image size
    name='car_damage_finetuned'    # A new name for the output folder
)

Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/FINAL_NEW_DATASET/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_damage_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspe

In [ ]:
# Install the ultralytics library
!pip install ultralytics -q

from ultralytics import YOLO
from IPython.display import Image, display

# --- 1. LOAD YOUR CUSTOM-TRAINED MODEL ---
# Replace 'best.pt' with the path to your model file if it's in a different folder
model = YOLO('/content/amira.pt')


# --- 2. DEFINE THE IMAGE YOU WANT TO CHECK ---
# IMPORTANT: Replace 'my_car_image.jpg' with the name of your uploaded image
image_to_check = '/content/photo_2025-09-14_22-52-17.jpg'


# --- 3. RUN THE PREDICTION ---
# The 'save=True' argument will save a new image with the bounding boxes drawn on it
results = model.predict(source=image_to_check, save=True)


# --- 4. EXTRACT AND PRINT TEXT RESULTS ---
print("\n--- Text Results ---")
for result in results:
    # The 'boxes' attribute contains the detected bounding boxes
    boxes = result.boxes

    # Check if any boxes were detected
    if len(boxes) == 0:
        print("No objects detected.")
        continue

    # Loop through each detected box
    for box in boxes:
        # Get the class ID (e.g., 0, 1, 2)
        class_id = int(box.cls[0])

        # Get the class name from the model's names dictionary
        class_name = model.names[class_id]

        # Get the confidence score
        confidence = float(box.conf[0])

        # Print the results in a user-friendly format
        print(f"Detected: {class_name} with confidence {confidence:.2f}")

# --- 5. DISPLAY THE VISUAL RESULT ---
# This part remains the same to show the image with boxes
print("\n--- Visual Result ---")
try:
    # Find the latest prediction folder to display the correct image
    import glob
    latest_predict_folder = sorted(glob.glob('runs/detect/predict*'))[-1]
    display(Image(filename=f'{latest_predict_folder}/{image_to_check}'))
except (FileNotFoundError, IndexError):
    print("Could not display the result image. Please check the 'runs/detect/' folder.")


image 1/1 /content/photo_2025-09-14_22-52-17.jpg: 320x640 1 dirt, 41.1ms
Speed: 2.2ms preprocess, 41.1ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /content/runs/detect/predict8

--- Text Results ---
Detected: dirt with confidence 0.79

--- Visual Result ---
Could not display the result image. Please check the 'runs/detect/' folder.
